# 一、简介
- parameter efficient fine tuning翻译为参数高效率微调，和全参数微调相对应
- 参考：https://zhuanlan.zhihu.com/p/707573525
- LoRA参考：https://zhuanlan.zhihu.com/p/688993851
- 代码可参考hf（除了下面介绍的，还有其他方法）：https://github.com/huggingface/peft
## 1. 为什么需要PEFT
- LLM模型参数量越来越大，训练一次需要耗费几亿美金。所以大部分都训练不起
- 因此很多小公司或者个人在开源LLM上微调，但是即使微调，也是大部分都负担不起的
- PEFT能节省大量微调所需资源，并且效果也很好
- 这种方式冻结大部分预训练参数，只微调一小部分参数

# 二、基于Adapter Tuning的方法
## 1. 原始架构
- 针对特定下游任务，在原始Transformer层中添加一个Adapter层
- 微调时固定原始所有参数，只训练新增的Adapter和LayerNorm的参数
<p align="center">
  <img src="./imgs/v2-42bb08dd821b120db744e1b82710ba80_b.png" width="300">
  <span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>
  <img src="./imgs/v2-89829b9356b7ed9fe826d9c15e215b7c_b.png" width="300">
</p>
<p align="center">
  <em>Image 1: 总体架构</em>
  <span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>
  <em>Image 2: Adapter内的架构</em>
</p>

## 2. Adapter Fusion
- 多任务微调时，针对每个任务分别用一个Adapter微调。
- 出煤Adapter还额外增加Adapter Fusion模块
- 在微调某一个任务时，需要固定原始模型参数和其他Adapter的参数；Adapter Fusion

<p align="center">
  <img src="./imgs/v2-2ce248ede2322257052f4f114f2735e2_720w.jpg" width="300">
  <span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>
  <img src="./imgs/v2-c9e5083409dab6db80a852eeee2bbbff_720w.jpg" width="300">
</p>
<p align="center">
  <em>Image 1: 总体架构</em>
  <span>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</span>
  <em>Image 2: Adapter内的架构</em>
</p>

## 3. Adapter-Drop
- 在Adapter fusion基础上，由于增加太多额外层，推理效率下降。
- 因此，在较低层的transformer就是用原始网络，不额外增加层

<p align="center">
  <img src="./imgs/v2-1a4d788eb753e8a3ee681acc1840e00b_720w.jpg" width="500">
</p>
<p align="center">
  <em>Image 1: 总体架构</em>
</p>

<p align="center">
  <img src="./imgs/v2-e3afa0803f7c606a0da73615e89dee9f_720w.jpg" width="500">
</p>
<p align="center">
  <em>Image 1: 不同任务使用不同的prefix</em>
</p>

# 三、基于soft-prompt的方法
- hard prompt就是人为手工构造的prompt，构建好prompt后，再prompt数据上做全量微调
- soft prompt是

<p align="center">
  <img src="./imgs/v2-f0f73329bf1e086f3545ac4d4283f398_720w.jpg" width="800">
</p>
<p align="center">
  <em>Image 1: 微调（fine-tuning）、prompt微调（soft-prompt），prompt设计（hard prompt）</em>
</p>


## 1. Prefix-tuning
- 在input前面（第一层前面）添加一个可以训练的prefix；在之后的每一层k、v前面添加prefix
- 不同任务使用不同的prefix，相同任务使用同一个prefix
- 在微调阶段，固定pretrained model参数，训练这个prefix
- 为了防止不稳定，不是直接训练prefix，而是训练一个MLP，这个MLP再生成prefix
- 部署时，直接抛弃MLP，只需要保留prefix

<p align="center">
  <img src="./imgs/v2-ff30ca83d5e2bcef2dc1e400e7bfda10_720w.jpg" width="800">
</p>
<p align="center">
  <em>Image 1: 总体架构</em>
</p>

## 2. Prompt-tuning
- 和prefix-tuning唯一不同的就是：直接训练prefix
- 因为研究者发现，在参数量很大时，稳定性的问题不存在了
- 并且prefix参数越多，效果越好

<p align="center">
  <img src="./imgs/v2-581ca4f87575033f2e31de7e35f4d7e9_720w.jpg" width="800">
</p>
<p align="center">
  <em>Image 1: 左：fine-tuning全量微调；右：prompt-tuning</em>
</p>

### (1). prompt-tuning实现

In [ ]:
import torch
import torch.nn as nn

class SoftEmbedding(nn.Module):
    def __init__(self, 
                wte: nn.Embedding,
                n_tokens: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to 
        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens, 
                                                                               random_range, 
                                                                               initialize_from_vocab))
            
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
            
    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [ ]:
from transformers import AutoConfig, AdamW, AutoTokenizer, AutoModel
import torch
import torch.nn as nn

n_tokens = 20
initialize_from_vocab = True
tokenizer = AutoTokenizer.from_pretrained("nezha-base-wwm")
config = AutoConfig.from_pretrained("nezha-base-wwm", num_labels=num_class)
config.output_hidden_states = True  # 需要设置为true才输出
model = AutoModel.from_pretrained(model_path, config=config)
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)
inputs = tokenizer("May the force be", return_tensors="pt")

# need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
# even though it does not matter what you pad input_ids with, it's just to make HF happy
inputs['input_ids'] = torch.cat([torch.full((1,n_tokens), 50256), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1), inputs['attention_mask']], 1)

outputs = model(**inputs)

## 3. P-tuning
- 在prompt-tuning的基础上重新添加了LSTM-MLP的网络编码soft-prompt
- 相当于重写了一个可学习embedding层

<p align="left">
  <img src="./imgs/f2a641f10fe24cb8b3775e103d35f51f~tplv-k3u1fbpfcp-zoom-in-crop-mark_1512_0_0_0.jpg" width="1200">
</p>
<p align="left">
  <em>Image 1: 左：相当于hard prompt；右：p-tuning</em>
</p>

# 四、 基于LoRA（Low-Rank Adaptation）的方法
- Adapter-tuning缺点：额外增加了计算延迟，因为不同层是串行的。
- Prompt-tuning缺点：并不好训练、微调后的性能和可训练参数数量不成正相关；prefix会占用token数，使得输出token数减少

## 1. LoRA
- LoRA（Low-Rank Adaptation）：和Adapter-tuning类似，都是新增加参数来微调。但是使用一个旁路（而不是串行）
- LoRA的思想是：目前的网络是过参数化（over-parameter）的，即weights矩阵实际上是一个低秩矩阵（大部分行/列向量可以有少部分的行/列向量线性表出），所以其实不需要那么多参数。因此将weights矩阵m×n拆分成m×r和r×n的两个小矩阵相乘。r就是一个内在秩。
- 微调时，原始参数固定，只训练新增的参数。并且将最终两个输出相加
- 原始论文只在attention上加了LoRA
- 另外需要特别说明的是：
    - LoRA的参数初始化时，A或者B必须某一个为全0，这样保证第一轮参数更新时，输出和原始模型输出一致。

<p align="center">
  <img src="./imgs/v2-840b7b18c8e5a4a61c7e625c550ed013_720w.jpg" width="500">
</p>
<p align="center">
  <em>Image 1: LoRA</em>
</p>


## 2. QLoRA（Quantilization Low-Rank Adaptation）
- 即量化和LoRA的结合，在微调和部署时，使用低精度